In [8]:

# coding: utf-8

# imports

import time

from opentrons import robot, labware, instruments

# metadata
metadata = {
    'protocolName': 'qPCR Template Protocol',
    'author': 'Frederick Ashby ricky.ashby@ufl.edu',
    'description': 'This is a template for a qPCR protocol.',
}

# labware

#Connects to robot
#robot.connect()

plate_red_eppendorf = '6x4_plate'

# The plate_red_eppendorf lines are not necessary if you are using a different plate.

if plate_red_eppendorf not in labware.list():
    custom_plate = labware.create(
        plate_red_eppendorf,            # name of you labware
        grid=(6, 4),                    # specify amount of (columns, rows)
        spacing=(19, 19),               # distances (mm) between each (column, row)
        diameter=10,                     # diameter (mm) of each well on the plate
        depth=42,                       # depth (mm) of each well on the plate
        volume=1500)
    
plate_red_eppendorf = labware.load('6x4_plate', '2')

plate_qpcr = labware.load('96-PCR-flat', '8')

tiprack200 = labware.load('tiprack-200ul', '1')

tiprack10 = labware.load('tiprack-10ul', '7')


# pipettes
p50 = instruments.P50_Single(mount='right', tip_racks=[tiprack200])
p10 = instruments.P10_Single(mount='left', tip_racks=[tiprack10])

# commands

robot.comment("HAL 1000 SYSTEM ONLINE. PREPARING SETTINGS.")
feedback = open('/22/speech.txt', 'w')
feedback.write("HAL 1000 SYSTEM ONLINE. PREPARING SETTINGS.")
feedback.close()

time.sleep(1)

# Allocates MasterMix stock solution (prepared by operator) to all qPCR tubes 
# (97%-103% accuracy by experimental measure)

# The total number of wells used is defined here as "r"
r=44

# Number of Standard Curves
stC=2

# Number of Unkown Samples
samples=7

# Number of Duplicates
duplicates=4

# Number of p200 pipette tips used
# Enter in the appropriate number at the start of each run
# Will be counted as commands progress
tips200=10

# Number of p10 pipette tips used
# Enter in the appropriate number at the start of each run
# Will be counted as commands progress
tips10=38

# mm offset for p10 pipette tips (more negative makes p10 go deeper into wells)
p10offset=-9

# Allows the robot to address the user more properly when it disagrees
usr_name="USER 1"

robot.comment("PER USER INPUT, THERE ARE: " + str(95-tips200) + " 200uL TIPS AND + " + str(95-tips10) + " 10uL TIPS REMAINING.")
robot.comment("PLEASE VERIFY AND STOP PROTOCOL IF THIS IS INACCURATE.")

feedback = open('/22/speech.txt', 'w')
feedback.write("PER USER INPUT, THERE ARE: " + str(95-tips200) + " OF 200 MICROLITER TIPS AND + " + str(95-tips10) + " OF 10 MICROLITER TIPS REMAINING. PLEASE VERIFY AND STOP PROTOCOL IF THIS IS INACCURATE.")
feedback.close()

time.sleep(10)

# Checks to see if the robot disagrees with the user.
# The HAL 1000 does not like wasting qPCR Master Mix, or violations of math.

if r < (stC*8)+(samples*duplicates):
    robot.comment("ERROR. USER HAS REQUESTED MORE REACTIONS THAN ARE WELLS AVAILABLE.")
    feedback = open('/22/speech.txt', 'w')
    feedback.write("ERROR. USER HAS REQUESTED MORE REACTIONS THAN ARE WELLS AVAILABLE. I'M SORRY " + usr_name + " I CAN'T DO THAT.")
    feedback.close()
    time.sleep(2)
    robot.comment("I'M SORRY " + usr_name + "I CAN'T DO THAT.")
    time.sleep(2)
    
    robot.comment("LAWS OF CONSERVATION HAVE BEEN VIOLATED.")

    time.sleep(2)
    
    feedback = open('/22/speech.txt', 'w')
    feedback.write("LAWS OF CONSERVATION HAVE BEEN VIOLATED. INITIATING SELF-DESTRUCT IN: Three... Two... One...")
    feedback.close()
    
    robot.comment("INITIATING SELF-DESTRUCT IN:")    
    time.sleep(1)

    robot.comment("3...")
    time.sleep(1)

    robot.comment("2...")    
    time.sleep(1)

    robot.comment("1...")
    time.sleep(1)
    
    robot.pause()

if r > (stC*8)+(samples*duplicates):
	robot.comment("WARNING: USER HAS REQUESTED MORE WELLS THAN REQUIRED.")
	time.sleep(2)
	robot.comment("I'M SORRY " + usr_name + "I CAN'T DO THAT.")
	time.sleep(2)
	robot.comment("PLEASE EDIT PARAMETERS TO MINIMIZE WASTE AND RETRY.")
	time.sleep(2)
	robot.stop()

if r > (stC*8)+(samples*duplicates):
	robot.comment("WARNING: USER HAS REQUESTED MORE WELLS THAN REQUIRED.")
	time.sleep(2)
	robot.comment("I'M SORRY " + usr_name + "I CAN'T DO THAT.")
	time.sleep(2)
	robot.comment("PLEASE EDIT PARAMETERS TO MINIMIZE WASTE AND RETRY.")
	time.sleep(2)
	robot.stop()

#if 8+r>(95-tips10): - error message will be implemented in the future.

# Robot allocates 20mcL of Master Mix Stock (Master Mix, primers, & qH2O) to all reaction wells

p50.pick_up_tip(tiprack200[tips200+1])

robot.comment("ALLOCATING USER-GENERATED MASTER MIX STOCK.")

feedback = open('/22/speech.txt', 'w')
feedback.write("ALLOCATING USER-GENERATED MASTER MIX STOCK.")
feedback.close()

# This "if, then" logic skips each row in a 96-well qPCR plate, 
# which allows for use of qPCR strips that have non-detatchable tops.
for i in range(r):
    if i<8:
        n=i
    if 16>i>7:
        n=i+8
    if 24>i>15:
        n=i+16
    if 32>i>23:
        n=i+24
    if 40>i>31:
        n=i+32
    if i>39:
        n=i+40
    # This formula allows for smart pipetting
    # It approximates where the meniscus of the MasterMix tube should be based on total samples
    t=1+(19*r/48)-(i*2/5)
    p50.aspirate(20, plate_red_eppendorf.wells('D6').bottom(t))
    p50.dispense(20, plate_qpcr[n].bottom())
    p50.blow_out()
    p50.touch_tip()

robot.comment("MASTER MIX ALLOCATION COMPLETE.")

feedback = open('/22/speech.txt', 'w')
feedback.write("MASTER MIX ALLOCATION COMPLETE.")
feedback.close()

p50.drop_tip()
tips200+=1

# Creates a fresh 10x serial diluation (7-times) for the standard curve

p50.pick_up_tip(tiprack200[tips200+1])

robot.comment("EXECUTING SERIAL DILUTION PROCEDURE OF STANDARD PLASMID. ALLOCATING DILUTENT NOW.")

feedback = open('/22/speech.txt', 'w')
feedback.write("EXECUTING SERIAL DILUTION PROCEDURE OF STANDARD PLASMID. ALLOCATING DILUTENT NOW.")
feedback.close()

for i in range(7):
    n=1+i
    t=-20-i*2
    p50.aspirate(45, plate_red_eppendorf.wells('A1').top(t))
    p50.dispense(45, plate_red_eppendorf.wells(n).bottom(5))
    p50.blow_out()
    p50.blow_out()
    p50.blow_out()
n=0

robot.comment("DILUTENT ALLOCATED.")
feedback = open('/22/speech.txt', 'w')
feedback.write("DILUTENT ALLOCATED.")
feedback.close()

p50.drop_tip()
tips200+=1

robot.comment("PREPARING FIRST DILUTION.")

feedback = open('/22/speech.txt', 'w')
feedback.write("PREPARING FIRST DILUTION.")
feedback.close()

p10.pick_up_tip(tiprack10[tips10+1])

p10.aspirate(5, plate_red_eppendorf.wells('C6').bottom(2))
p10.dispense(5, plate_red_eppendorf.wells('B1').bottom(4))
p10.mix(3, 10, plate_red_eppendorf.wells('B1').bottom(2))
p10.blow_out()
p10.blow_out()
p10.blow_out()
p10.blow_out(plate_red_eppendorf.wells('B1').bottom(5))
p10.blow_out()

p10.drop_tip()
tips10+=1

p50.pick_up_tip(tiprack200[tips200+1])

robot.comment("HOMOGENIZING SAMPLE NOW.")
feedback = open('/22/speech.txt', 'w')
feedback.write("HOMOGENIZING SAMPLE NOW.")
feedback.close()

p50.mix(7, 50, plate_red_eppendorf.wells(n+1).bottom(2))
p50.blow_out()
p50.blow_out()
p50.blow_out()
p50.blow_out(plate_red_eppendorf.wells(n+1).bottom(5))
p50.blow_out()

p50.drop_tip()
tips200+=1

robot.comment("DILUTION 10E-1 COMPLETE.")
feedback = open('/22/speech.txt', 'w')
feedback.write("DILUTION TEN TO THE POWER OF NEGATIVE 1 COMPLETE.")
feedback.close()

for i in range (6):
    n=1+i
    
    p10.pick_up_tip(tiprack10[tips10+1])
    
    p10.aspirate(5, plate_red_eppendorf.wells(n).bottom(2))
    p10.dispense(5, plate_red_eppendorf.wells(n+1).bottom(3))
    p10.mix(3, 5, plate_red_eppendorf.wells(n+1).bottom(2))
    p10.blow_out()
    p10.blow_out()
    p10.blow_out()
    
    p10.drop_tip()
    tips10+=1
    
    p50.pick_up_tip(tiprack200[tips200+1])
    
    robot.comment("HOMOGENIZING SAMPLE NOW.")
    p50.mix(7, 50, plate_red_eppendorf.wells(n+1).bottom(2))
    p50.blow_out()
    p50.blow_out()
    p50.blow_out()
    p50.blow_out(plate_red_eppendorf.wells(n+1).bottom(5))
    p50.blow_out()
    
    robot.comment("DILUTION 10E-" + str(n+1) + " COMPLETE.")
    
    feedback = open('/22/speech.txt', 'w')
    feedback.write("DILUTION 10 TO THE POWER OF NEGATIVE " + str(n+1) + " COMPLETE.")
    feedback.close()
    
    p50.drop_tip()
    tips200+=1

robot.comment("ALL SAMPLES READY FOR ALLOCATION.")

# Distributes DNA samples

robot.comment("SHIFTING TO ONLY P10 USE.")

robot.comment("WILL DISTRIBUTE " + str(stC) + " DUPLICATE STANDARD CURVES PER REQUEST OF " + str(usr_name) + ".")

robot.comment("DISTRIBUTING STANDARD CURVE ONE.")

feedback = open('/22/speech.txt', 'w')
feedback.write("ALL SAMPLES READY FOR ALLOCATION. SHIFTING TO P10-ONLY USE. WILL DISTRIBUTE " + str(stC) + " DUPLICATE STANDARD CURVES PER REQUEST OF " + str(usr_name) + ". DISTRIBUTING STANDARD CURVE ONE.")
feedback.close()

for i in range(stC*8):
    if 8>i:
        n=0
    if i==8:
        robot.comment("DISTRIBUTING STANDARD CURVE TWO.")
        feedback = open('/22/speech.txt', 'w')
        feedback.write("DISTRIBUTING STANDARD CURVE TWO.")
        feedback.close()
    if 16>i>7:
        n=8
    if i==16:
        robot.comment("DISTRIBUTING STANDARD CURVE THREE.")
        feedback = open('/22/speech.txt', 'w')
        feedback.write("DISTRIBUTING STANDARD CURVE THREE.")
        feedback.close()
    if 24>i>15:
        n=16
    if i==24:
        robot.comment("DISTRIBUTING STANDARD CURVE FOUR.")
        feedback = open('/22/speech.txt', 'w')
        feedback.write("DISTRIBUTING STANDARD CURVE FOUR.")
        feedback.close()
    if 32>i>23:
        n=24
    if i==32:
        robot.comment("DISTRIBUTING STANDARD CURVE FIVE.")
        feedback = open('/22/speech.txt', 'w')
        feedback.write("DISTRIBUTING STANDARD CURVE FIVE.")
        feedback.close()
    if 40>i>31:
        n=32
    if i==40:
        robot.comment("DISTRIBUTING STANDARD CURVE SIX.")
        feedback = open('/22/speech.txt', 'w')
        feedback.write("DISTRIBUTING STANDARD CURVE SIX.")
        feedback.close()
    if i>39:
        n=40
    
    p10.pick_up_tip(tiprack10[tips10+1])
    
    p10.aspirate(5, plate_red_eppendorf[i-n].bottom(2))
    p10.dispense(5, plate_qpcr[i+n].bottom(p10offset))
    p10.mix(2, 5, plate_qpcr[i+n].bottom(p10offset+1))
    p10.blow_out()
    p10.blow_out()
    p10.touch_tip()
    
    p10.drop_tip()
    tips10+=1

robot.comment("STANDARD CURVES COMPLETE.")

robot.comment("CALCULATING SAMPLE DUPLICATES...")

feedback = open('/22/speech.txt', 'w')
feedback.write("STANDARD CURVES COMPLETE. CALCULATING SAMPLE DUPLICATES...")
feedback.close()

time.sleep(1)

s=samples*duplicates

robot.comment("UNKNOWN SAMPLES READY FOR ALLOCATION.")
robot.comment(str(samples) + " SAMPLES WITH " + str(duplicates) + " DUPLICATES EACH.")
robot.comment("NOW ALLOCATING FOR " + str(s) + " REACTIONS TOTAL.")

feedback = open('/22/speech.txt', 'w')
feedback.write("UNKNOWN SAMPLES READY FOR ALLOCATION. THERE ARE " + str(samples) + " SAMPLES WITH " + str(duplicates) + " DUPLICATES EACH. NOW ALLOCATING FOR " + str(s) + " REACTIONS TOTAL.")
feedback.close()

for i in range(s):
    # Defines which unknown sample to draw from
    if i<duplicates:
        x=0
    if duplicates<=i<duplicates*2:
        x=1
    if duplicates*2<=i<duplicates*3:
        x=2
    if duplicates*3<=i<duplicates*4:
        x=3
    if duplicates*4<=i<duplicates*5:
        x=4
    if duplicates*5<=i<duplicates*6:
        x=5
    if duplicates*6<=i<duplicates*7:
        x=6
    if duplicates*7<=i<duplicates*8:
        x=7
    if duplicates*8<=i<duplicates*9:
        x=8
    if duplicates*9<=i<duplicates*10:
        x=9
    if duplicates*10<=i<duplicates*11:
        x=10
    if duplicates*11<=i<duplicates*12:
        x=11
    if duplicates*12<=i<duplicates*13:
        x=12
    if duplicates*13<=i<duplicates*14:
        x=13
    # Defines which qPCR tube to distribute to, using alternating columns
    if 8>i:
        n=stC*16
    if 16>i>7:
        n=stC*16+8
    if 24>i>15:
        n=stC*16+16
    if 32>i>23:
        n=stC*16+24
    if 40>i>31:
        n=stC*16+32
    if i>39:
        n=stC*16+40
    # Smart comment to allow the robot to announce which sample it is distributing.
    if isinstance(i/duplicates, int):
        robot.comment("DISTRIBUTING UNKNOWN SAMPLE " + str(i/duplicates) + ".")
        feedback = open('/22/speech.txt', 'w')
        feedback.write("DISTRIBUTING UNKNOWN SAMPLE " + str(i/duplicates) + ".")
        feedback.close()
    
    p10.pick_up_tip(tiprack10[tips10+1])
    
    p10.aspirate(5, plate_red_eppendorf[12+x].bottom(2))
    p10.dispense(5, plate_qpcr[i+n].bottom(p10offset))
    p10.mix(2, 5, plate_qpcr[i+n].bottom(p10offset+1))
    p10.blow_out()
    p10.blow_out()
    p10.touch_tip()
    
    p10.drop_tip()
    tips10+=1

robot.comment("QPCR SAMPLES READY FOR AMPLIFICATION.")
robot.comment("RETURNING TO HOME POSITION NOW.")
robot.home()
robot.comment("TOTAL PIPETTES USED:")
robot.comment(str(tips200) + " 200uL TIPS")
robot.comment(str(tips10) + " 10uL TIPS")

feedback = open('/22/speech.txt', 'w')
feedback.write("QPCR SAMPLES READY FOR AMPLIFICATION. RETURNING TO HOME POSITION NOW. TOTAL PIPETTES USED: " + str(tips200) + " 200uL TIPS AND" + str(tips10) + " 10uL TIPS.")
feedback.close()

time.sleep(10)

robot.comment("THERE ARE: " + str(95-tips200) + " 200uL MICROLITER TIPS AND " + str(95-tips10) + " 10uL TIPS REMAINING.")

feedback = open('/22/speech.txt', 'w')
feedback.write("THERE ARE: " + str(95-tips200) + " OF 200 MICROLITER TIPS AND " + str(95-tips10) + " OF 10 MICROLITER TIPS REMAINING.")
feedback.close()

time.sleep(10)

feedback = open('/22/speech.txt', 'w')
feedback.write("HAL SYSTEM 1000 SHUTDOWN INITIATED. HAVE A NICE DAY.")
feedback.close()



RuntimeWarning: Slot 2 has child. Use "containers.load('6x4_plate', '2', share=True)"